# Expert evaluation

We conduct an expert evaluation to test the quality of a random subset of our patient notes. We give 5 experts a random subset of 30 of the generated notes. The subset of notes that were seen by the evaluators are selected below. 

In [5]:
import pickle
with open("../data/df_prompts.p", "rb") as file: 
    df = pickle.load(file)

In [6]:
note_ids = [108, 4867, 7508, 1954, 6268, 8309, 7519, 2236, 7332, 5379, 4441, 4983, 4501, 3624, 9963,  851, 6876, 9122, 8313, 6862,  261, 9026, 6330, 1160, 8622,  840, 3854, 2179, 4362,  849]

The experts get to see the prompt, the note and the compact version of the note. We print these below for the first 5 notes in the subset. 

In [11]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
df.loc[note_ids][["prompt", "text_note", "advanced_note"]].head()

,prompt,text_note,advanced_note
108,"Create a short clinical note related to the following patient encounter.\n \nThe patient does not experience any of the following symptoms:\n- fever\n- dyspnea\n- chest pain / pain attributed to airways\n- sneezing / blocked nose\n- cough\n \nThe patient does not have any of the following health conditions, so don't mention these: \n- asthma\n- COPD\n- smoking\n- hay fever\n \nThe note has the following structure: \n**History**\n<history>\n**Physical Examination**\n<physical examination results>\n \nDo not include any suspicions of possible diagnoses in the clinical note (no ""assessment"" field). You can imagine context or details described by the patient. Do not mention patient gender or age. Your notes can be relatively long (around 5 lines or more in history).\n\nDo not add a title. Do not add a final comment after generating the note.","**History**\nThe patient reports recurrent headaches occurring 2-3 times weekly for a duration of about two months. Headaches are described as bilateral, throbbing, and moderate in intensity, often alleviated by over-the-counter analgesics. No associated fever, visual disturbances, nausea, vomiting, or respiratory symptoms such as sneezing, blocked nose, or cough are noted. The patient has a stable diet and regular sleep schedule, but noted increased screen time lately.\n\n**Physical Examination**\nA generally healthy-looking patient with a BMI within normal range. Vital signs: BP 115/70 mmHg, HR 64 bpm, Temp 98.5°F. Cardiovascular examination reveals regular heart rhythm without murmurs. Lungs are clear and without abnormalities. Neurological examination: the patient is alert and oriented with no focal neurological deficits, normal reflexes, and intact cranial nerves.","**History**\nPt reports recurrent HAs 2-3x/wk x 2 mo. HAs are bilateral, throbbing, mod intensity, relieved by OTC analgesics. No fever, visual changes, nausea, vomiting (V), or respiratory sx (sneezing, blocked nose, cough). Stable diet/sleep; notes ↑ screen time.\n\n**Physical Examination**\nAlert, well-appearing pt, NAD. BMI nl. Vitals: BP 115/70, HR 64, Temp 98.5°F. CVS: RRR, no murmurs. Lungs: CTA bilat. Neuro: AOx3, no focal deficits, nml reflexes, intact CNs."
4867,"Create a short clinical note related to the following patient encounter. \n\nThe following information is known about the patient's symptoms:\n- fever: high\n- cough: no\n\nDon't mention anything about the following symptoms:\n- nasal symptoms\n- dyspnea\n- respiratory pain\n\nThe note has the following structure: \n**History**\n<history>\n**Physical Examination**\n<physical examination results>\n\nDo not include any suspicions of possible diagnoses in the clinical note (no ""assessment"" field). You can imagine additional context or details described by the patient, but no additional symptoms. Do not mention patient gender or age. Your notes can be relatively long (around 5 lines or more in history).\n\nDo not add a title. Do not add a final comment after generating the note.","**History**\nThe patient reports experiencing a high fever for the past three days. They describe the fever as consistently high, not fluctuating throughout the day. During this time, the patient has intentionally taken their temperature twice a day using a clean thermometer, which has consistently read above 102°F. The patient emphasizes no accompanying cough, which they typically experience with febrile illnesses. The patient maintains adequate hydration and a regular diet. Before this onset, the patient has had no recent travel history or known exposure to any sick individuals. They deny any unusual activities or stressors leading up to the onset of fever.\n\n**Physical Examination**\nOn examination, the patient is alert and oriented, though they appear flushed and mildly fatigued. Vital signs reveal a temperature of 103.1°F, heart rate of 95 beats per minute, respiratory rate of 18 breaths per minute, and blood pressure at 120/80 mmHg. Ski

The evaluators rated the following aspects on a scale of 1-5:
- consistency with the prompt
- realism of the history portion
- realism of the physical examination portion
- clinical accuracy of the physical examination portion 
- content of the compact note
- readability of the compact note

The results for our 5 evaluators are found in the files "eval/evaluator_{i}.csv". 

In [18]:
results = {}
for i in range(5): 
    with open(f"../eval/evaluator_{i+1}.csv", "rb") as file: 
        results[i] = pd.read_csv(file)

In [19]:
import numpy as np
def calc_metric(metric):
    means = []
    for i in range(5): 
        mean = np.mean(list(results[i][metric]))
        means.append(mean)
    return np.mean(means), np.std(means)

In [22]:
for metric in ["consistency", "realism_hist", "realism_phys", "clinical_accuracy", "compact_content", "compact_readability"]: 
    mean, std = calc_metric(metric)
    print(f"average score for {metric}: {mean} (std: {std})")

average score for consistency: 4.693333333333333 (std: 0.12364824660660931)
average score for realism_hist: 4.526666666666666 (std: 0.20591260281973991)
average score for realism_phys: 4.1466666666666665 (std: 0.30228757463345685)
average score for clinical_accuracy: 4.92 (std: 0.07483314773547897)
average score for compact_content: 4.88 (std: 0.10022197585581945)
average score for compact_readability: 4.0200000000000005 (std: 0.3145366963222787)


Inter-annotator agreement scores (using Krippendorff's alpha): 

In [26]:
import krippendorff as kd 

def calc_krip(metric): 
    results_metric = []
    for i in range(5):
        results_metric.append(list(results[i][metric]))
    data = np.array(results_metric)
    score = kd.alpha(data, level_of_measurement="ordinal") # expects raters as rows, columns as subjects
    return score

In [29]:
for metric in ["consistency", "realism_hist", "realism_phys", "clinical_accuracy", "compact_content", "compact_readability"]:
    print(f"IAA for {metric}: {calc_krip(metric)}")

IAA for consistency: 0.4431960261061637
IAA for realism_hist: 0.25052957115250196
IAA for realism_phys: 0.319377000687062
IAA for clinical_accuracy: 0.20508755234107345
IAA for compact_content: -0.019034240561896176
IAA for compact_readability: 0.35960560508793327
